# Natural frequency of a traffic signal 🚏

In this notebook we are going to find an approximate value of the frequency of a traffic signal. We will use the Rayleigh's method to approximate the signal to a one degree of freedom structure.

## Definitions

First of all, we will define the material properties (density $\rho$ and Young modulus $E$) and the sections. The mast has a constant cross section with a given area ($A$) and intertia modulus ($I$). Then, a constant linear density ($\rho A$) and constant stiffness ($EI$) is defined. The traffic signal on top is considered as a concentrated mass.

In [11]:
from math import pi, sqrt

steel_density = 7.7      # T/m3
young_modulus = 210*1e6  # kPa

class CircularSignal:
    def __init__(self, diameter, thickness):
        self.mass = steel_density * thickness * pi * 0.25 * diameter**2 * 1e-9  # T

class SquareSignal:
    def __init__(self, width, height, thickness):
        self.mass = steel_density * thickness * width * height * 1e-9  # T

class CircularSection:
    def __init__(self, diameter, thickness):
        self.linear_density = steel_density * thickness * pi * diameter * 1e-6                          # T/m
        self.stiffness = young_modulus * 1/64 * pi * (diameter**4 - (diameter-2*thickness)**4) * 1e-12  # kN*m2

class RectangularSection:
    def __init__(self, width, depth, thickness):
        self.linear_density = steel_density * thickness * 2 * (width+depth) * 1e-6                                     # T/m
        self.stiffness = young_modulus * 1/12 * (width*depth**3 - (width-2*thickness)*(depth-2*thickness)**3) * 1e-12  # kN*m2

 ## Rayleigh's method

The traffic signal is a structure clamped at the bottom and free at the top. Hence, a second order polynomial is choosen as shape function:
$$
\psi = \frac{z^2}{h^2} \\
\psi'' = \frac{2}{h^2}
$$
which satisfies the kynematic boundary conditions $\psi(0)=\psi'(0)=0$.

The frequency is obtained after some algebra using the Rayleigh's expressions:
\begin{align*}
m &= \int_0^h \rho A\psi^2 dz + \sum \psi_i^2 m_i = \frac{1}{5} \rho A h + m_1 \\
k &= {\displaystyle \int_0^h EI\psi''^2 dz} = 4 \frac{EI}{h^3}\\
w^2 &= \frac{k}{m}
\end{align*}

In [19]:
def NaturalFrequency(mast_section, signal_shape, height):
    m = mast_section.linear_density * height / 5 + signal_shape.mass
    k = 4 * mast_section.stiffness / height**3
    frequency = sqrt(k/m)           # rad/s
    nat_frequency = frequency/2/pi  # Hz
    return nat_frequency

## Calculations

In [23]:
mast = RectangularSection(width=80, depth=40, thickness=1)  # section dimensions in mm
signal = CircularSignal(diameter=600, thickness=1)          # section dimensions in mm

nat_frequency = NaturalFrequency(mast, signal, height=3)    # structure dimensions in m

print(f'Natural frequency: {nat_frequency:.1f} Hz')

Natural frequency: 4.1 Hz


In [22]:
mast = CircularSection(diameter=60, thickness=1)            # section dimensions in mm
signal = CircularSignal(diameter=600, thickness=1)          # section dimensions in mm

nat_frequency = NaturalFrequency(mast, signal, height=3)    # structure dimensions in m

print(f'Natural frequency: {nat_frequency:.1f} Hz')

Natural frequency: 4.6 Hz
